In [1]:
from models import Models
from Data_preprocessing import split_60 ,split_train_val_test ,load_data
from utils import save_data, split_train_test ,plot_scatter_hr ,plot_loss ,calculate_loss_hr
from custom_callbacks import EarlyStoppingIncreasingValLoss

from sklearn.preprocessing import MinMaxScaler


import pandas as pd
from IPython.display import display



from tensorflow.keras.models import save_model

from keras.optimizers import RMSprop ,Adam ,SGD ,Adagrad
from keras.callbacks import EarlyStopping

import json
import os
import time

import tensorflow as tf
tf.random.set_seed(1234)

2023-06-14 10:15:44.577440: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 10:15:44.916217: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.1/lib64:
2023-06-14 10:15:44.916249: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-14 10:15:44.945844: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-14 10:15:46.15

In [2]:
model_name = 'model2_hr'
NUM_EPOCH = 300
scaled = False

# lr = 5e-4
# optimizer = RMSprop(learning_rate=lr)
# loss = "mse"

In [3]:
df , ratio = load_data()
df

(5439, 1813, 1813)


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,hr,calculated_hr,DBP,SBP
0,0.022165,0.008058,-0.005694,-0.018277,-0.028401,-0.034445,-0.034832,-0.028548,-0.015653,0.002471,...,0.006408,-0.014542,-0.033742,-0.049071,-0.058827,-0.061907,70,99.23,75,111
1,0.002396,-0.001148,-0.004544,-0.007492,-0.009561,-0.010251,-0.009101,-0.005841,-0.000541,0.006277,...,0.023915,0.031500,0.035881,0.036621,0.033664,0.027309,85,86.31,86,133
2,-0.007708,-0.005857,-0.004098,-0.002622,-0.001656,-0.001300,-0.001325,-0.001099,0.000269,0.003553,...,-0.055547,-0.063149,-0.066136,-0.062301,-0.050670,-0.031967,59,57.83,87,139
3,-0.010544,-0.005010,0.000461,0.005682,0.010287,0.013727,0.015341,0.014503,0.010822,0.004327,...,-0.027160,-0.043717,-0.057046,-0.066155,-0.070509,-0.070022,86,68.22,80,146
4,-0.007411,-0.007268,-0.007012,-0.006467,-0.005435,-0.003734,-0.001218,0.002173,0.006360,0.011074,...,-0.023944,-0.009014,0.006739,0.022151,0.036009,0.047166,85,72.06,80,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9060,0.028062,0.013806,-0.000827,-0.014660,-0.026783,-0.036512,-0.043327,-0.046852,-0.046867,-0.043359,...,-0.031265,-0.024912,-0.017063,-0.008171,0.001152,0.010138,82,82.40,79,133
9061,0.054225,0.071998,0.084893,0.091340,0.090327,0.081546,0.065480,0.043417,0.017377,-0.010103,...,0.020310,0.008572,-0.002583,-0.012376,-0.020267,-0.026010,86,77.07,80,146
9062,0.045568,0.033181,0.017340,-0.000325,-0.017933,-0.033597,-0.045684,-0.052995,-0.054832,-0.050976,...,0.068069,0.077176,0.078598,0.072051,0.058184,0.038529,93,90.76,81,118
9063,-0.108834,-0.099573,-0.085189,-0.067508,-0.048111,-0.028148,-0.008312,0.011064,0.029816,0.047741,...,-0.067566,-0.058927,-0.048947,-0.037186,-0.023145,-0.006584,68,69.99,84,148


In [4]:
df_input = df.drop(['DBP','SBP','hr','calculated_hr'], axis=1)
df_target = df[['hr']]

display(df_target)

scaler = MinMaxScaler()
scaled_df_target = scaler.fit_transform(df_target)
print(df_input.shape)
print(scaled_df_target.shape)

if scaled == True:
    df_target = scaled_df_target
        
X_train, y_train, X_val, y_val, X_test, y_test = split_train_test(df_input ,df_target ,ratio)
    

,hr
0,70
1,85
2,59
3,86
4,85
...,...
9060,82
9061,86
9062,93
9063,68


(9065, 60)
(9065, 1)


In [5]:

lr_list = [5e-4 ,5e-5 ,5e-3]
loss_list = ['mse' , 'mae']

for loss in loss_list:
    for lr in lr_list:
        optimizer_list = [RMSprop(learning_rate=lr) ,Adam(learning_rate=lr) ,SGD(learning_rate=lr) ,Adagrad(learning_rate=lr)]
        for optimizer in optimizer_list:
            
            models = Models()
            model = models.import_model(model_name)
            model.summary()

            model.compile(loss=loss, optimizer=optimizer, metrics=['mse','mae'])
            
            early_stopping = EarlyStoppingIncreasingValLoss(patience=2, delta=0, check_interval=5, increase_threshold=5)
            history = model.fit(X_train, y_train, validation_split=0.2, epochs=NUM_EPOCH ,callbacks=[early_stopping])

            optimizer_name = optimizer.get_config()['name']

            result_name = f'{model_name}_lr:{lr}_NUM_EPOCH:{NUM_EPOCH}_optimizer:{optimizer_name}_loss:{loss}'
            folder_path = f'./hr_optimization_results/{result_name}'
            os.makedirs(folder_path)

            history_dict = history.history
            with open(f'{folder_path}/history.json', 'w') as f:
                json.dump(history_dict, f)

            results = model.evaluate(X_test, y_test)
            save_model(model, f"{folder_path}/model.h5")

            y_predicted = model.predict(X_test)

            if scaled == True:
                y_predicted_temp = scaler.inverse_transform(y_predicted)
                y_test_temp = scaler.inverse_transform(y_test)
            else:
                y_predicted_temp = y_predicted
                y_test_temp = y_test


            y_predicted_df = pd.DataFrame(y_predicted_temp)
            y_test_df = pd.DataFrame(y_test_temp)

            plot_scatter_hr(y_predicted_df,y_test_df ,folder_path)

            mae , mse = calculate_loss_hr(y_predicted_df,y_test_df)

            plot_loss(history_dict ,folder_path)

            out_dict = {
                    'model_name':[model_name],
                    'lr':[lr],
                    'epochs':[NUM_EPOCH],
                    'optimizer':[optimizer_name],
                    'loss':[loss],
                    'results':[results],
                    'mae':[mae],
                    'mse':[mse],
                    'time':[time.time()]

                    }

            save_data(out_dict, 'sheets/hr_model_optimizing.csv')

2023-06-14 10:15:48.982887: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-14 10:15:48.982910: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (atamach01): /proc/driver/nvidia/version does not exist
2023-06-14 10:15:48.983366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 60)            14880     
                                                                 
 dropout (Dropout)           (None, 60, 60)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 60, 240)          173760    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 60, 240)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 120)              144480    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 1

Epoch 39/300
136/136 [==============================] - 20s 147ms/step - loss: 124.4906 - mse: 124.4906 - mae: 9.1354 - val_loss: 122.7420 - val_mse: 122.7420 - val_mae: 9.1126
Epoch 40/300
136/136 [==============================] - 21s 152ms/step - loss: 124.6212 - mse: 124.6212 - mae: 9.1543 - val_loss: 122.7225 - val_mse: 122.7225 - val_mae: 9.0922
Epoch 41/300
136/136 [==============================] - 24s 174ms/step - loss: 124.8877 - mse: 124.8877 - mae: 9.1407 - val_loss: 122.6793 - val_mse: 122.6793 - val_mae: 9.0939
Epoch 42/300
136/136 [==============================] - 24s 176ms/step - loss: 124.5598 - mse: 124.5598 - mae: 9.1421 - val_loss: 122.8180 - val_mse: 122.8180 - val_mae: 9.1237
Epoch 43/300
136/136 [==============================] - 20s 148ms/step - loss: 124.7804 - mse: 124.7804 - mae: 9.1681 - val_loss: 122.6903 - val_mse: 122.6903 - val_mae: 9.0979
Epoch 44/300
136/136 [==============================] - 20s 148ms/step - loss: 124.6233 - mse: 124.6233 - mae: 9.15

136/136 [==============================] - 20s 148ms/step - loss: 106.9599 - mse: 106.9599 - mae: 8.3216 - val_loss: 103.4756 - val_mse: 103.4756 - val_mae: 8.2039
Epoch 86/300
136/136 [==============================] - 21s 152ms/step - loss: 105.4727 - mse: 105.4727 - mae: 8.2714 - val_loss: 103.5212 - val_mse: 103.5212 - val_mae: 8.0725
Epoch 87/300
136/136 [==============================] - 23s 165ms/step - loss: 105.8094 - mse: 105.8094 - mae: 8.2538 - val_loss: 100.7622 - val_mse: 100.7622 - val_mae: 7.9425
Epoch 88/300
136/136 [==============================] - 21s 152ms/step - loss: 105.5449 - mse: 105.5449 - mae: 8.2664 - val_loss: 102.6401 - val_mse: 102.6401 - val_mae: 8.0946
Epoch 89/300
136/136 [==============================] - 21s 153ms/step - loss: 106.0370 - mse: 106.0370 - mae: 8.2765 - val_loss: 100.1791 - val_mse: 100.1791 - val_mae: 8.0077
Epoch 90/300
136/136 [==============================] - 21s 154ms/step - loss: 106.4009 - mse: 106.4009 - mae: 8.2980 - val_loss

Epoch 132/300
136/136 [==============================] - 20s 146ms/step - loss: 93.8862 - mse: 93.8862 - mae: 7.7457 - val_loss: 101.2962 - val_mse: 101.2962 - val_mae: 7.9912
Epoch 133/300
136/136 [==============================] - 20s 147ms/step - loss: 94.3526 - mse: 94.3526 - mae: 7.7880 - val_loss: 97.7970 - val_mse: 97.7970 - val_mae: 7.8935
Epoch 134/300
136/136 [==============================] - 20s 148ms/step - loss: 93.8089 - mse: 93.8089 - mae: 7.7355 - val_loss: 99.0996 - val_mse: 99.0996 - val_mae: 7.9850
Epoch 135/300
136/136 [==============================] - 20s 150ms/step - loss: 94.2005 - mse: 94.2005 - mae: 7.7547 - val_loss: 99.4994 - val_mse: 99.4994 - val_mae: 7.8287
Epoch 136/300
136/136 [==============================] - 21s 155ms/step - loss: 93.9481 - mse: 93.9481 - mae: 7.7454 - val_loss: 98.8433 - val_mse: 98.8433 - val_mae: 7.9241
Epoch 137/300
136/136 [==============================] - 20s 148ms/step - loss: 93.9329 - mse: 93.9329 - mae: 7.7427 - val_loss:

...

Epoch 116/300
136/136 [==============================] - 35s 255ms/step - loss: 4273.9346 - mse: 4273.9346 - mae: 64.4169 - val_loss: 4261.5449 - val_mse: 4261.5449 - val_mae: 64.3340
Epoch 117/300
136/136 [==============================] - 36s 264ms/step - loss: 4271.4736 - mse: 4271.4736 - mae: 64.3978 - val_loss: 4258.7153 - val_mse: 4258.7153 - val_mae: 64.3120
Epoch 118/300
136/136 [==============================] - 28s 209ms/step - loss: 4268.4727 - mse: 4268.4727 - mae: 64.3742 - val_loss: 4255.9106 - val_mse: 4255.9106 - val_mae: 64.2902
Epoch 119/300
136/136 [==============================] - 32s 233ms/step - loss: 4265.5508 - mse: 4265.5508 - mae: 64.3516 - val_loss: 4253.1318 - val_mse: 4253.1318 - val_mae: 64.2686
Epoch 120/300
136/136 [==============================] - 29s 211ms/step - loss: 4263.0391 - mse: 4263.0391 - mae: 64.3318 - val_loss: 4250.3789 - val_mse: 4250.3789 - val_mae: 64.2472
Epoch 121/300
136/136 [==============================] - 27s 200ms/step - loss: 

136/136 [==============================] - 28s 204ms/step - loss: 4169.9277 - mse: 4169.9277 - mae: 63.6042 - val_loss: 4157.9741 - val_mse: 4157.9741 - val_mae: 63.5240
Epoch 161/300
136/136 [==============================] - 38s 278ms/step - loss: 4167.7817 - mse: 4167.7817 - mae: 63.5877 - val_loss: 4156.0498 - val_mse: 4156.0498 - val_mae: 63.5088
Epoch 162/300
136/136 [==============================] - 38s 281ms/step - loss: 4166.0591 - mse: 4166.0591 - mae: 63.5739 - val_loss: 4154.1396 - val_mse: 4154.1396 - val_mae: 63.4938
Epoch 163/300
136/136 [==============================] - 29s 213ms/step - loss: 4164.0640 - mse: 4164.0640 - mae: 63.5582 - val_loss: 4152.2461 - val_mse: 4152.2461 - val_mae: 63.4789
Epoch 164/300
136/136 [==============================] - 31s 227ms/step - loss: 4162.3462 - mse: 4162.3462 - mae: 63.5445 - val_loss: 4150.3672 - val_mse: 4150.3672 - val_mae: 63.4641
Epoch 165/300
136/136 [==============================] - 43s 321ms/step - loss: 4160.4370 - ms

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fbc9cceb0d0> (for post_execute):


KeyboardInterrupt: 